In [3]:
import os
import pympi

def add_rttm_tier_to_eaf(
    eaf_folder,
    rttm_folder,
    output_folder=None,
    new_tier_name="RTTM"
):
    """
    Parcourt un dossier de fichiers EAF, trouve pour chacun le RTTM associé
    (même base de nom), et ajoute dans l'EAF une tier avec des annotations vides
    correspondant aux timecodes du RTTM.
    
    :param eaf_folder: Chemin vers le dossier contenant les fichiers .eaf
    :param rttm_folder: Chemin vers le dossier contenant les fichiers .rttm
    :param output_folder: Dossier où sauvegarder les EAF modifiés.
                          Si None, on écrase l'original dans eaf_folder.
    :param new_tier_name: Nom de la tier qui sera créée dans l'EAF.
    """

    if output_folder is None:
        # Si aucun dossier de sortie n'est spécifié, on écrase les EAF originaux.
        # Sinon, on peut créer un nouveau dossier.
        output_folder = eaf_folder
    
    os.makedirs(output_folder, exist_ok=True)

    # Liste des EAF
    for eaf_file in os.listdir(eaf_folder):
        if not eaf_file.lower().endswith(".eaf"):
            continue
        
        base_name, _ = os.path.splitext(eaf_file)
        
        eaf_path = os.path.join(eaf_folder, eaf_file)
        rttm_path = os.path.join(rttm_folder, base_name + ".rttm")
        
        if not os.path.isfile(rttm_path):
            print(f"[Info] Pas de RTTM pour {eaf_file}. Fichier attendu : {rttm_path}")
            continue
        
        # Charger l'EAF
        eaf_obj = pympi.Elan.Eaf(eaf_path)

        # Vérifier si la tier existe déjà, sinon la créer
        if new_tier_name not in eaf_obj.get_tier_names():
            eaf_obj.add_tier(new_tier_name)

        # Lire le RTTM et extraire les segments
        segments = parse_rttm_segments(rttm_path)

        # Ajouter des annotations vides pour chaque segment
        for (start_s, end_s) in segments:
            start_ms = int(start_s * 1000)
            end_ms = int(end_s * 1000)

            # Ajout d'une annotation vide
            eaf_obj.add_annotation(
                new_tier_name,
                start_ms,
                end_ms,
                ""  # Annotation vide
            )

        # Sauvegarde de l'EAF
        output_eaf_path = os.path.join(output_folder, eaf_file)
        eaf_obj.to_file(output_eaf_path)
        print(f"[OK] EAF modifié : {output_eaf_path}")


def parse_rttm_segments(rttm_file):
    """
    Lit un fichier RTTM et retourne une liste de segments (start, end) en secondes,
    extraits des lignes 'SPEAKER'.
    """
    segments = []
    with open(rttm_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            
            parts = line.split()
            # Format RTTM classique (9-10 champs) : 
            # 0       1       2       3       4        5      6 ...
            # SPEAKER fileID  chan   start_s  dur_s    <...>  speakerID ...
            # On vérifie qu'on a au moins 5 champs, et que parts[0] == "SPEAKER"
            if parts[0].upper() == "SPEAKER" and len(parts) >= 5:
                start_s = float(parts[3])
                dur_s = float(parts[4])
                end_s = start_s + dur_s
                segments.append((start_s, end_s))
    
    return segments


if __name__ == "__main__":
    dossier_eaf = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf"
    dossier_rttm = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/VAD/raw/recordings/converted/synth/wav"
    dossier_sortie = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD" 

    add_rttm_tier_to_eaf(
        eaf_folder=dossier_eaf,
        rttm_folder=dossier_rttm,
        output_folder=dossier_sortie,
        new_tier_name="RTTM"
    )


[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw0552.eaf
[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw0070.eaf
[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw1162.eaf
[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw1176.eaf
[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw0494.eaf
[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw0978.eaf
[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw0388.eaf
[OK] EAF modifié : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD/bdmw1